In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import Dataset, DataLoader
from skimage import io, transform

cudnn.benchmark = True
plt.ion()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data_transforms = { 
    'train': transforms.Compose([
        transforms.ToPILImage(),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'valid': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

In [4]:
data_dir = '/content/drive/MyDrive/oxford-102-flowers/'

In [5]:
datasets = {}
datasets['train'] = []
datasets['valid'] = []
datasets['test'] = []

# classes = []

def load_paths(x):
  with open(f'{data_dir}/{x}.txt', 'r') as f:
    for line in f:
      t = line.split()
      t[0] = data_dir + t[0]
      t[1] = int(t[1])

      datasets[x].append(t)

load_paths('train')
load_paths('valid')
load_paths('test')

In [6]:
class FlowersDataset(Dataset):

  def __init__(self, samples, transform = None):
    self.samples = samples
    self.transform = transform

  def __len__(self):
    return len(self.samples)

  def __getitem__(self, idx):
    if torch.is_tensor(idx):
      idx = idx.to_list()

    img_name = os.path.join(self.samples[idx][0])
    image = io.imread(img_name)
    if self.transform:
      image = self.transform(image)

    label = self.samples[idx][1]

    # sample = (image, label)

    return image, label

In [7]:
flower_dataset = {x: FlowersDataset(samples = datasets[x], transform = data_transforms[x]) for x in ['train', 'valid', 'test']}

flower_dataloaders = {x: torch.utils.data.DataLoader(flower_dataset[x], batch_size=8, shuffle=True, num_workers=2) for x in ['train', 'valid', 'test']}

dataset_sizes = {x: len(flower_dataset[x]) for x in ['train', 'valid', 'test']}

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [8]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25, datald=flower_dataloaders):

  since = time.time()

  best_model_wts = copy.deepcopy(model.state_dict())
  best_acc = 0.0

  for epoch in range(num_epochs):
      print(f'Epoch {epoch}/{num_epochs - 1}')
      print('-' * 10)

      # Each epoch has a training and validation phase
      for phase in ['train', 'valid']:
          if phase == 'train':
              model.train()  # Set model to training mode
          else:
              model.eval()   # Set model to evaluate mode

          running_loss = 0.0
          running_corrects = 0

          # Iterate over data.
          for inputs, labels in datald[phase]:

              
              inputs = inputs.to(device)
              
              labels = labels.to(device)

              # zero the parameter gradients
              optimizer.zero_grad()

              # forward
              # track history if only in train
              with torch.set_grad_enabled(phase == 'train'):
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

                  # backward + optimize only if in training phase
                  if phase == 'train':
                      loss.backward()
                      optimizer.step()

              # statistics
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)
          if phase == 'train':
              scheduler.step()

          epoch_loss = running_loss / dataset_sizes[phase]
          epoch_acc = running_corrects.double() / dataset_sizes[phase]

          print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

          # deep copy the model
          if phase == 'valid' and epoch_acc > best_acc:
              best_acc = epoch_acc
              best_model_wts = copy.deepcopy(model.state_dict())

      print()

  time_elapsed = time.time() - since
  print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
  print(f'Best val Acc: {best_acc:4f}')

  # load best model weights
  model.load_state_dict(best_model_wts)
  return model

In [9]:
def predict(model, criterion, optimizer, scheduler, num_epochs=25, datald=flower_dataloaders):

  since = time.time()
  best_acc = 0.0


  for epoch in range(num_epochs):
      print(f'Epoch {epoch}/{num_epochs - 1}')
      print('-' * 10)

      for phase in ['test']:

          model.eval()   # Set model to evaluate mode

          running_loss = 0.0
          running_corrects = 0

          # Iterate over data.
          for inputs, labels in datald[phase]:

              
              inputs = inputs.to(device)
              
              labels = labels.to(device)

              # zero the parameter gradients
              optimizer.zero_grad()

              # forward
              with torch.set_grad_enabled(False):
                  outputs = model(inputs)
                  _, preds = torch.max(outputs, 1)
                  loss = criterion(outputs, labels)

              # statistics
              running_loss += loss.item() * inputs.size(0)
              running_corrects += torch.sum(preds == labels.data)

          epoch_loss = running_loss / dataset_sizes[phase]
          epoch_acc = running_corrects.double() / dataset_sizes[phase]

          print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

          # deep copy the model
          if epoch_acc > best_acc:
              best_acc = epoch_acc
              best_model_wts = copy.deepcopy(model.state_dict())

      # print()

  time_elapsed = time.time() - since
  print(f'Testing complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
  print(f'Best test Acc: {best_acc:4f}')

In [10]:
def imshow(inp, title=None):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated


In [11]:
model_ft = models.resnet34(pretrained=True)
num_ftrs = model_ft.fc.in_features
# Here the size of each output sample is set to 2.

model_ft.fc = nn.Linear(num_ftrs, 102)

model_ft = model_ft.to(device)

criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [12]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 4.4146 Acc: 0.0735
valid Loss: 3.1967 Acc: 0.3490

Epoch 1/24
----------
train Loss: 3.1139 Acc: 0.3441
valid Loss: 1.8870 Acc: 0.6333

Epoch 2/24
----------
train Loss: 2.1528 Acc: 0.5892
valid Loss: 1.1645 Acc: 0.7422

Epoch 3/24
----------
train Loss: 1.4965 Acc: 0.7186
valid Loss: 0.8229 Acc: 0.8088

Epoch 4/24
----------
train Loss: 1.1702 Acc: 0.7931
valid Loss: 0.6527 Acc: 0.8559

Epoch 5/24
----------
train Loss: 1.0355 Acc: 0.8176
valid Loss: 0.5604 Acc: 0.8627

Epoch 6/24
----------
train Loss: 0.7417 Acc: 0.8784
valid Loss: 0.4721 Acc: 0.8922

Epoch 7/24
----------
train Loss: 0.6287 Acc: 0.9010
valid Loss: 0.4159 Acc: 0.9088

Epoch 8/24
----------
train Loss: 0.5516 Acc: 0.9137
valid Loss: 0.4269 Acc: 0.9029

Epoch 9/24
----------
train Loss: 0.5548 Acc: 0.9118
valid Loss: 0.3916 Acc: 0.9059

Epoch 10/24
----------
train Loss: 0.5143 Acc: 0.9294
valid Loss: 0.3996 Acc: 0.9078

Epoch 11/24
----------
train Loss: 0.5192 Acc: 0.9196
valid Loss

In [13]:
predict(model_ft, criterion, optimizer_ft, exp_lr_scheduler, num_epochs=1)

Epoch 0/0
----------
test Loss: 0.4654 Acc: 0.8888

Testing complete in 1m 5s
Best test Acc: 0.888762


In [14]:
model_2 = models.resnet34(pretrained=True)
num_ftrs = model_2.fc.in_features
# Here the size of each output sample is set to 102.
model_2.fc = nn.Linear(num_ftrs, 102)

model_2 = model_2.to(device)

# Observe that all parameters are being optimized
optimizer_2 = optim.SGD(model_2.parameters(), lr=0.005, momentum=0.9)

In [15]:
model_2 = train_model(model_2, criterion, optimizer_2, exp_lr_scheduler,
                       num_epochs=25)

Epoch 0/24
----------
train Loss: 4.3731 Acc: 0.0980
valid Loss: 3.2552 Acc: 0.2990

Epoch 1/24
----------
train Loss: 2.8429 Acc: 0.3275
valid Loss: 2.2123 Acc: 0.4667

Epoch 2/24
----------
train Loss: 2.2032 Acc: 0.4353
valid Loss: 1.7138 Acc: 0.5824

Epoch 3/24
----------
train Loss: 1.8529 Acc: 0.5186
valid Loss: 1.7604 Acc: 0.5804

Epoch 4/24
----------
train Loss: 1.5560 Acc: 0.5833
valid Loss: 1.8471 Acc: 0.6108

Epoch 5/24
----------
train Loss: 1.3870 Acc: 0.6157
valid Loss: 1.1393 Acc: 0.7333

Epoch 6/24
----------
train Loss: 1.1774 Acc: 0.6637
valid Loss: 1.4956 Acc: 0.6882

Epoch 7/24
----------
train Loss: 1.1122 Acc: 0.6941
valid Loss: 1.3608 Acc: 0.7039

Epoch 8/24
----------
train Loss: 0.9707 Acc: 0.7441
valid Loss: 1.0811 Acc: 0.7667

Epoch 9/24
----------
train Loss: 0.8907 Acc: 0.7539
valid Loss: 0.9573 Acc: 0.7794

Epoch 10/24
----------
train Loss: 0.7946 Acc: 0.7990
valid Loss: 1.0274 Acc: 0.7745

Epoch 11/24
----------
train Loss: 0.8634 Acc: 0.7676
valid Loss

In [16]:
predict(model_2, criterion, optimizer_2, exp_lr_scheduler, num_epochs=1)

Epoch 0/0
----------
test Loss: 0.8720 Acc: 0.8099

Testing complete in 1m 4s
Best test Acc: 0.809888


**Transformations used:**

*    RandomResizedCrop(224)
*    RandomHorizontalFlip()

**Val Accuracy:**

* lr = 0.01 -> Accuracy = 0.92
* lr = 0.05 -> Accuracy = 0.84

**Test Accuracy:**

* lr = 0.01 -> Accuracy = 0.88
* lr = 0.05 -> Accuracy = 0.80
